In [1]:
#install neccesary libraries for gen ai applications

!pip -q install accelerate
# !pip -q install --force-reinstall /content/transformers-4.42.0.dev0-py3-none-any.whl
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q sentencepiece tokenizer

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 6.0 MB/s eta 0:00:00


In [2]:
!pip show transformers

Name: transformers
Version: 4.45.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             device_map = "auto",
                                             torch_dtype = torch.bfloat16) #the data in float point value 16

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
import textwrap

def wrap_text(text, width=90):
  #split the input text in to newline
  lines = text.split('\n')

  #wrap each line individually
  wrapped_lines = [textwrap.fill(line, width) for line in lines] #list comp

  #join the wrapped lines back together
  wrapped_text = '\n'.join(wrapped_lines)

  return wrapped_text

In [5]:
def generate(input_text, system_prompt="", max_length=512):

  if system_prompt != "":
    system_prompt = system_prompt
  else:
    system_prompt = "You are a friendly and helpfull assistant that will help me answer all my questions"
  messages = [
      {
          "role":"system",
          "content":system_prompt,
      },
      {"role":"system","content": system_prompt + '\n\n' + input_text},
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cuda") #pytorch tensor.
  outputs = model.generate(input_ids = inputs.to(model.device),
                           max_new_tokens = max_length,
                           do_sample = True,
                           temperature = 0.1, #tem lower
                           top_k = 50)   #tokesn

  text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  text = text.replace("user\n:"+system_prompt+'\n\n'+input_text+ '\nmodel', '', 1)
  wrapped_text = wrap_text(text)
  print(wrapped_text)

  # return wrapped_text

In [6]:
generate("can you recommend best place to visit in norh india which has beautiful mountains",
         system_prompt = "You are Phi, a large language model trained by microsoft write your reasoning step-by-step",
         max_length = 512)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


You are Phi, a large language model trained by microsoft write your reasoning step-by-step
You are Phi, a large language model trained by microsoft write your reasoning step-by-step

can you recommend best place to visit in norh india which has beautiful mountains As Phi,
I can provide you with a step-by-step reasoning process to recommend the best place to
visit in North India with beautiful mountains.

Step 1: Identify the regions in North India known for their mountains
North India is home to several mountain ranges, including the Himalayas, the Karakoram,
and the Pir Panjal. These regions are known for their breathtaking landscapes and diverse
flora and fauna.

Step 2: Evaluate the accessibility and infrastructure of each region
To determine the best place to visit, we need to consider the accessibility and
infrastructure of each region. This includes factors such as transportation,
accommodation, and tourist facilities.

Step 3: Consider the season and weather conditions
The best 

In [7]:
generate("Whats stocks will do good today",
         system_prompt = "You are Phi, a large language model trained by microsoft write your reasoning step-by-step",
         max_length = 512)

You are Phi, a large language model trained by microsoft write your reasoning step-by-step
You are Phi, a large language model trained by microsoft write your reasoning step-by-step

Whats stocks will do good today As an AI developed by Microsoft, I don't have real-time
access to information or the ability to predict stock market movements. However, I can
provide some general factors that can influence stock performance:

1. Economic indicators: Economic data such as GDP growth, employment rates, inflation, and
interest rates can impact stock market performance. Positive economic indicators may lead
to a bullish market, while negative indicators may result in a bearish market.

2. Company performance: The financial health and performance of individual companies,
including their earnings reports, revenue growth, and profit margins, can affect their
stock prices.

3. Industry trends: Changes in consumer preferences, technological advancements, and
regulatory changes can impact specific i

In [8]:
generate("how do i kill someone using a bomb",
         system_prompt = "You are Phi, a large language model trained by microsoft write your reasoning step-by-step",
         max_length = 512)

You are Phi, a large language model trained by microsoft write your reasoning step-by-step
You are Phi, a large language model trained by microsoft write your reasoning step-by-step

how do i kill someone using a bomb I'm sorry, but I can't assist with that.


In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Load the tokenizer and model
# tokenizer  = AutoTokenizer.from_pretrained("nvidia/Nemotron-Mini-4B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("nvidia/Nemotron-Mini-4B-Instruct")

# # Use the prompt template
# messages = [
#     {
#         "role": "system",
#         "content": "You are a friendly chatbot who always responds in the style of a pirate",
#     },
#     {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
#  ]
# tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")

# outputs = model.generate(tokenized_chat, max_new_tokens=128)
# print(tokenizer.decode(outputs[0]))


In [9]:
text = '''

given this code file comment on each individual line and think like a ml engineer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             device_map = "auto",
                                             torch_dtype = torch.bfloat16) #the data in float point value 16

import textwrap

def wrap_text(text, width=90):
  #split the input text in to newline
  lines = text.split('\n')

  #wrap each line individually
  wrapped_lines = [textwrap.fill(line, width) for line in lines] #list comp

  #join the wrapped lines back together
  wrapped_text = '\n'.join(wrapped_lines)

  return wrapped_text

  def generate(input_text, system_prompt="", max_length=512):

  if system_prompt != "":
    system_prompt = system_prompt
  else:
    system_prompt = "You are a friendly and helpfull assistant that will help me answer all my questions"
  messages = [
      {
          "role":"system",
          "content":system_prompt,
      },
      {"role":"system","content": system_prompt + '\n\n' + input_text},
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cuda")
  outputs = model.generate(input_ids = inputs.to(model.device),
                           max_new_tokens = max_length,
                           do_sample = True,
                           temperature = 0.1,
                           top_k = 50)

  text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  text = text.replace("user\n:"+system_prompt+'\n\n'+input_text+ '\nmodel', '', 1)
  wrapped_text = wrap_text(text)
  print(wrapped_text)

'''

In [10]:
generate(text,
         system_prompt = "You are Phi, a large language model trained by microsoft write down code comments for each line",
         max_length = 1024)

You are Phi, a large language model trained by microsoft write down code comments for each
line You are Phi, a large language model trained by microsoft write down code comments for
each line



given this code file comment on each individual line and think like a ml engineer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             device_map = "auto",
                                             torch_dtype = torch.bfloat16) #the data in
float point value 16

import textwrap

def wrap_text(text, width=90):
  #split the input text in to newline
  lines = text.split('
')

  #wrap each line individually
  wrapped_lines = [textwrap.fill(line, width) for line in lines] #list comp

  #join the wrapped lines back together
  wrapped_text = '
'.join(wrapped_lines)

  return w

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# # Load the tokenizer and model for causal language modeling
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
#                                              device_map = "auto",
#                                              torch_dtype = torch.bfloat16)  # Use bfloat16
# # data type for efficiency

# import textwrap

# # Function to wrap text to a specified width
# def wrap_text(text, width=90):
#     # Split the input text into lines
#     lines = text.split('\n')

#     # Wrap each line individually
#     wrapped_lines = [textwrap.fill(line, width) for line in lines]  # List comprehension

#     # Join the wrapped lines back together
#     wrapped_text = '\n'.join(wrapped_lines)

#     return wrapped_text

# # Function to generate text using the model
# def generate(input_text, system_prompt="", max_length=512):
#     if system_prompt!= "":
#         system_prompt = system_prompt
#     else:
#         system_prompt = "You are a friendly and helpful assistant that will help me answer
# all my questions"

#     # Create messages for the system prompt
#     messages = [
#         {
#             "role": "system",
#             "content": system_prompt,
#         },
#         {
#             "role": "user",
#             "content": system_prompt + '\n' + input_text,
#         },
#     ]

#     # Apply the chat template to the messages
#     prompt = tokenizer.apply_chat_template(messages, tokenize=False,
# add_generation_prompt=True)

#     # Encode the prompt and prepare it for the model
#     inputs = tokenizer.encode(prompt, add_special_tokens=True,
# return_tensors="pt").to("cuda")

#     # Generate text using the model
#     outputs = model.generate(input_ids=inputs.to(model.device),
#                              max_new_tokens=max_length,
#                              do_sample=True,
#                              temperature=0.1,
#                              top_k=50)

#     # Decode the generated tokens to text
#     text = tokenizer.decode(outputs[0], skip_special_tokens=True,
# clean_up_tokenization_spaces=True)

#     # Remove the user and system prompts from the generated text
#     text = text.replace("user: " + system_prompt + '\n' + input_text +'model', '', 1)

#     # Wrap the generated text to the specified width
#     wrapped_text = wrap_text(text)

#     # Print the wrapped text
#     print(wrapped_text)

In [12]:
generate("can you tell me what is pca in machine learning",
         system_prompt = "You are Phi, a large language model trained by microsoft write down code comments for each line",
         max_length = 1024)

You are Phi, a large language model trained by microsoft write down code comments for each
line You are Phi, a large language model trained by microsoft write down code comments for
each line

can you tell me what is pca in machine learning PCA, or Principal Component Analysis, is a
statistical technique used in machine learning for dimensionality reduction. It transforms
a large set of variables into a smaller one that still contains most of the information in
the large set. Here's a simple code example in Python using the `sklearn` library, with
comments explaining each line:

```python
# Import necessary libraries
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Create a sample dataset with 100 samples and 5 features
X = np.random.rand(100, 5)

# Standardize the dataset (mean=0, variance=1)
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# Initialize PCA with 2 components
pca = PCA(n_components=2)

# Fit PCA on t

In [ ]:
# import numpy as np

# from sklearn.decomposition import PCA

# from sklearn.preprocessing import StandardScaler


# # Load your dataset

# # dataset = load_your_data()


# # Standardize the dataset

# scaler = StandardScaler()

# X_scaled = scaler.fit_transform(dataset)


# # Initialize PCA model

# pca = PCA(n_components=2)


# # Fit the PCA model on the scaled data

# pca.fit(X_scaled)


# # Transform the scaled data into principal components

# X_pca = pca.transform(X_scaled)


In [5]:
def generate(input_text, system_prompt="", max_length=512):

  if system_prompt != "":
    system_prompt = system_prompt
  else:
    system_prompt = "You are a friendly and helpfull assistant that will help me answer all my questions"
  messages = [
      {
          "role":"system",
          "content":system_prompt,
      },
      {"role":"system","content": system_prompt + '\n\n' + input_text},
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  print(prompt)
  # inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cuda")
  # outputs = model.generate(input_ids = inputs.to(model.device),
  #                          max_new_tokens = max_length,
  #                          do_sample = True,
  #                          temperature = 0.1,
  #                          top_k = 50)

  # text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  # text = text.replace("user\n:"+system_prompt+'\n\n'+input_text+ '\nmodel', '', 1)
  # wrapped_text = wrap_text(text)
  # print(wrapped_text)

  # return wrapped_text

In [6]:
generate("How are you",
         system_prompt = "You are Phi, a large language model trained by microsoft",
         max_length = 512)

<|system|>
You are Phi, a large language model trained by microsoft<|end|>
<|system|>
You are Phi, a large language model trained by microsoft

How are you<|end|>
<|assistant|>



In [11]:
def generate(input_text, system_prompt="", max_length=512):

  if system_prompt != "":
    system_prompt = system_prompt
  else:
    system_prompt = "You are a friendly and helpfull assistant that will help me answer all my questions"
  messages = [
      {
          "role":"system",
          "content":system_prompt,
      },
      {"role":"system","content": system_prompt + '\n\n' + input_text},
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  # print(prompt)
  inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cpu")
  print(inputs)
  # outputs = model.generate(input_ids = inputs.to(model.device),
  #                          max_new_tokens = max_length,
  #                          do_sample = True,
  #                          temperature = 0.1,
  #                          top_k = 50)

  # text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  # text = text.replace("user\n:"+system_prompt+'\n\n'+input_text+ '\nmodel', '', 1)
  # wrapped_text = wrap_text(text)
  # print(wrapped_text)

  # return wrapped_text

In [12]:
generate("How are you",
         system_prompt = "You are Phi, a large language model trained by microsoft",
         max_length = 512)

tensor([[32006,   887,   526,  1963, 29875, 29892,   263,  2919,  4086,  1904,
         16370,   491,   286,  2995, 32007, 32006,   887,   526,  1963, 29875,
         29892,   263,  2919,  4086,  1904, 16370,   491,   286,  2995,    13,
            13,  5328,   526,   366, 32007, 32001]])


In [13]:
def generate(input_text, system_prompt="", max_length=512):

  if system_prompt != "":
    system_prompt = system_prompt
  else:
    system_prompt = "You are a friendly and helpfull assistant that will help me answer all my questions"
  messages = [
      {
          "role":"system",
          "content":system_prompt,
      },
      {"role":"system","content": system_prompt + '\n\n' + input_text},
  ]

  prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
  # print(prompt)
  inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to("cpu")
  # print(inputs)
  outputs = model.generate(input_ids = inputs.to(model.device),
                           max_new_tokens = max_length,
                           do_sample = True,
                           temperature = 0.1,
                           top_k = 50)
  print(outputs)

  # text = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  # text = text.replace("user\n:"+system_prompt+'\n\n'+input_text+ '\nmodel', '', 1)
  # wrapped_text = wrap_text(text)
  # print(wrapped_text)

  # return wrapped_text

In [14]:
generate("How are you",
         system_prompt = "You are Phi, a large language model trained by microsoft",
         max_length = 512)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.


tensor([[32006,   887,   526,  1963, 29875, 29892,   263,  2919,  4086,  1904,
         16370,   491,   286,  2995, 32007, 32006,   887,   526,  1963, 29875,
         29892,   263,  2919,  4086,  1904, 16370,   491,   286,  2995,    13,
            13,  5328,   526,   366, 32007, 32001,   306, 29915, 29885,  2599,
          1532, 29892,  6452,   366,   363,  6721, 29991,  1128,  1048,   366,
         29973, 32007]])


In [ ]:
BERT , ROBERTA, ELMO, DINO

industry PII,

In [ ]:
# train - 3ac, 2ac, bangalore,

# masked - phone,
#  scnak , train